# Experiments & Model Training

### Load Q&A Dataset and Extend with Scenario Answers

In [15]:
from Shared.MLModelUtils.Chatbot.ScenarioUtils import ScenarioUtils
from Shared.MLModelUtils.Chatbot.LogousChatbotScenarioBase import LogousChatbotScenarioBase
from Shared.MLModelUtils.Chatbot.BotUtils import BotUtils
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

lower_map = {
    ord(u'I'): u'ı',
    ord(u'İ'): u'i',
}

data = pd.read_csv('Data/qa_dataset.csv', index_col=0)
data.reset_index(inplace=True)
data["QUESTION"] = data["QUESTION"].str.translate(lower_map).str.lower()

# Add possible scenario q/a
custom = pd.DataFrame({'QUESTION': [
                                    'izin girişi yapmak istiyorum', 
                                    'izin girmek istiyorum', 
                                    'izin istiyorum'
                                   ],
                       'ANSWER':   [
                                    ScenarioUtils.RUN_SCENARIO +':0',  # Run scenario with the ID:0
                                    ScenarioUtils.RUN_SCENARIO +':0',
                                    ScenarioUtils.RUN_SCENARIO +':0'
                                   ],                               
                      })                      

data = pd.concat([data, custom])
data.reset_index(inplace=True)
data.drop(['index'], axis=1,inplace=True)

### Word Embedding Model Training

#### Prepare W2V Training Data

In [16]:
# Read word2vec training data
w2vdata = pd.read_csv('Data/w2v_dataset.csv', index_col=0)
w2vdata.reset_index(inplace=True)
w2vdata["DESC"] = w2vdata["DESC"].str.translate(lower_map).str.lower()

# Combine word2vec training data with scenario questions
if (custom is not None) and (len(custom)>0):
    try:
        custom.drop(['ANSWER'], axis=1,inplace=True)
        custom["QUESTION"] = custom["QUESTION"].str.translate(lower_map).str.lower()
        custom.rename(columns={'QUESTION':'DESC'}, inplace=True)    
    except:
        pass

w2v_training_data = pd.concat([w2vdata, custom])
w2v_training_data.reset_index(inplace=True)
w2v_training_data.drop(['index'], axis=1,inplace=True)
w2v_training_data[-10:]

,DESC
71,teşekkürler biz teşekkür ederiz
72,hoşçakal teşekkür ederiz
73,yardım gerekli
74,yardıma ihtiyacım var
75,izin girişi yapmak istiyorum
76,izin girmek istiyorum
77,izin istiyorum
78,izin girişi yapmak istiyorum
79,izin girmek istiyorum
80,izin istiyorum


#### Train Word Embedding Model

In [17]:
import gensim
from gensim.models import Word2Vec
import numpy as np
from Shared.MLModelUtils.Chatbot.BotUtils import BotUtils

utils = BotUtils(None, globals())

# Preprocess all dataset
context = []
for i in range(0,np.shape(w2v_training_data)[0]):       
    context.append(utils.preprocess(w2v_training_data.iloc[i].DESC))

# train word2vec model for all corpus
model = Word2Vec(context, min_count=1)

questions = data["QUESTION"]
answers = data["ANSWER"]
[signatures, answers] = utils.calculate_answer_hashes(questions[:59], answers[:59], model) 

# Save external model files for production
np.save('sample_model_signatures.npy', signatures)
np.save('sample_model_answers.npy', answers)
np.save('sample_model_questions.npy', questions)
model.save('sample_model_embedding.bin')

In [18]:
data

,ANSWER,QUESTION
0,Cevap:1,disiplin yönetmeliği etik hat süreci
1,Cevap:2,etik bildirim tutanağı
2,Cevap:3,disiplin yönetmeliği etik bildirim tutanağı
3,Cevap:4,etik kurul çalışma yönetmeliği
4,Cevap:5,organizasyon şeması
5,Cevap:6,yeni işe başlayanlar katılanlar kimlerdir
6,Cevap:7,iş ilanları açıp pozisyonlar neler
7,Cevap:8,açık pozisyonlara içeriden başvuru süreci
8,Cevap:9,nasıl staj yapılıyor stajer stajyer nasıl alın...
9,Cevap:10,ücret yönetimi avans talep süreci
